In [16]:
import requests
import json

In [2]:
def deal_json_invaild(data):
    data = data.replace(' ','')
    data = data.replace("\n", "\\n").replace("\r", "\\r").replace("\n\r", "\\n\\r") \
        .replace("\r\n", "\\r\\n") \
        .replace("\t", "\\t")
    data = data.replace('":"', '&&testPassword1&&')\
        .replace('":', '&&testPassword2&&')\
        .replace('","', "$$testPassword$$")\
        .replace('{"', "@@testPassword@@")\
        .replace('"}', "**testPassword**")
    # print(data)

    data = data.replace('"', r'\"')\
        .replace('&&testPassword1&&', '":"').replace('&&testPassword2&&','":').replace('$$testPassword$$', '","').replace('@@testPassword@@', '{"').replace('**testPassword**', '"}')
    # print(data)
    return data

In [4]:
import oss2
ali_user_name = 'LTAI5tRAbpW9MNjDDnDAVCsq'
ali_pwd = '7RguIff46JiTrh43HkULgbXZPiIVqu'
auth = oss2.Auth(ali_user_name, ali_pwd)
# yourEndpoint填写Bucket所在地域对应的Endpoint。以华东1（杭州）为例，Endpoint填写为https://oss-cn-hangzhou.aliyuncs.com。
# 填写Bucket名称。
bucket = oss2.Bucket(auth, 'oss-ap-southeast-1.aliyuncs.com', 'aistormy2023')
oss_url_prefix = "https://aistormy2023.oss-ap-southeast-1.aliyuncs.com/"

file_name='test_cv.pdf'
content = open(file_name, 'rb').read()
ret = bucket.put_object(file_name, content)

# v2.0

In [19]:
url = 'http://114.248.215.173:32040'
api_list = [
"/test",
"/recruit/job/register",
"/recruit/job/query",
"/recruit/account/register",
"/recruit/account/query",
"/recruit/account/task/fetch",
"/recruit/account/task/report",
"/recruit/candidate/filter",
"/recruit/candidate/chat",
"/recruit/candidate/result"
]

In [20]:
api_dict = {}
for api in api_list:
    sp = api[1:].split('/')
    api_key = sp[-1]
    if len(sp)>1:
        api_key+=f'_{sp[-2]}'
    api_dict[api_key]=api
    print(f'{api_key}\t\t\t{api}')

test			/test
register_job			/recruit/job/register
query_job			/recruit/job/query
register_account			/recruit/account/register
query_account			/recruit/account/query
fetch_task			/recruit/account/task/fetch
report_task			/recruit/account/task/report
filter_candidate			/recruit/candidate/filter
chat_candidate			/recruit/candidate/chat
result_candidate			/recruit/candidate/result


## job op

In [43]:
r = requests.post(url+api_dict['register_job'],json={
    'platformType': 'Boss',
    'platformID': 'minsheng-kefu-manual-id',
    'jobName': '民生银行-外呼客服',
    'jd': '''银行岗位jd
招聘公司：知名大型商业银行
岗位名称：外呼客服
岗位职责：通过银行外呼系统对已经使用信用卡产生消费账单的用户，让用户办理分期还款，减轻还款压力。
岗位要求：年龄18-35周岁，大专及以上学历，普通话标准，良好的沟通表达能力
薪资福利：综合薪资8000-20000，上不封顶，入职缴纳北京五险一金，享受法定节假日
工作时间：早9晚6''',
    'robotApi': '/vision/chat/receive/message/minsheng/v1'
})
r.text

'{"ret": 0, "msg": "success", "data": {"jobID": "job_Boss_minsheng-kefu-manual-id"}}'

In [44]:
r = requests.post(url+api_dict['query_job'],json={
    'platformType': 'Boss',
    'platformID': 'minsheng-kefu-manual-id'
})
r.text

'{"ret": 0, "msg": "success", "data": {"jobID": "job_Boss_minsheng-kefu-manual-id"}}'

In [45]:
job_id="job_Boss_minsheng-kefu-manual-id"

In [83]:
r = requests.post(url+api_dict['register_job'],json={
    'platformType': 'Boss',
    'platformID': 'unicom-kefu-manual-id',
    'jobName': '联通-客服专员',
    'jd': '''招聘公司：知名国企
岗位名称：客服专员
岗位职责：负责接听热线，解答解决用户的查询、咨询和业务办理及疑难问题
岗位要求：38岁以下，中专及以上学历，普通话标准，良好的沟通表达能力
薪资福利：综合薪资4000-7000，入职缴纳五险一金，法定节假日三薪
工作时间：每天8小时不加班''',
    'robotApi': '/vision/chat/receive/message/unicom/v1'
})
r.text

'{"ret": 0, "msg": "success", "data": {"jobID": "job_Boss_unicom-kefu-manual-id"}}'

In [84]:
shijiazhuang_job_id="job_Boss_unicom-kefu-manual-id"

## account op

In [79]:
r = requests.post(url+api_dict['register_account'],json={
    'platformType': 'Boss',
    'platformID': '27175761',
    'jobs': [job_id]
})
r.text

'{"ret": 0, "msg": "success", "data": {"accountID": "account_Boss_27175761"}}'

In [80]:
r = requests.post(url+api_dict['register_account'],json={
    'platformType': 'Boss',
    'platformID': '611930762',
    'jobs': [job_id]
})
r.text

'{"ret": 0, "msg": "success", "data": {"accountID": "account_Boss_611930762"}}'

In [67]:
r = requests.post(url+api_dict['query_account'],json={
    'platformType': 'Boss',
    'platformID': '611930762',
})
r.text

'{"ret": 0, "msg": "success", "data": {"accountID": "account_Boss_611930762"}}'

In [85]:
r = requests.post(url+api_dict['register_account'],json={
    'platformType': 'Boss',
    'platformID': '19887133',
    'jobs': [shijiazhuang_job_id],
    'desc': '陈旭彤'
})
r.text

'{"ret": 0, "msg": "success", "data": {"accountID": "account_Boss_19887133"}}'

In [86]:
r = requests.post(url+api_dict['register_account'],json={
    'platformType': 'Boss',
    'platformID': '4431156',
    'jobs': [shijiazhuang_job_id],
    'desc': '田润和'
})
r.text

'{"ret": 0, "msg": "success", "data": {"accountID": "account_Boss_4431156"}}'

In [62]:
account_id="account_Boss_27175761"

## task op

In [63]:
r = requests.post(url+api_dict['fetch_task'],json={
    'accountID': account_id,
})
r.text

'{"ret": 0, "msg": "success", "data": {"task": [{"jobID": "job_Boss_minsheng-kefu-manual-id", "taskType": "batchTouch", "helloSum": 50, "timeMount": []}]}}'

In [64]:
r = requests.post(url+api_dict['fetch_task'],json={
    'accountID':'account_Boss_611930762',
})
r.text

'{"ret": 0, "msg": "success", "data": {"task": [{"jobID": "job_Boss_minsheng-kefu-manual-id", "taskType": "batchTouch", "helloSum": 50, "timeMount": []}]}}'

In [31]:
r = requests.post(url+api_dict['report_task'],json={
    'accountID': account_id,
    'taskStatus': [{
        'taskType': 'batchTouch',
        'details': {
            'candidateList': ['mock-589791359']
        }
    }]
})
r.text

'{"ret": 0, "msg": "success", "data": {"status": "ok"}}'

In [55]:
r = requests.post(url+api_dict['fetch_task'],json={
    'accountID': account_id,
})
r.text

'{"ret": 0, "msg": "success", "data": {"task": [{"helloSum": 50, "taskType": "batchTouch", "timeMount": [{"time": "09:00", "mount": 25}, {"time": "16:00", "mount": 25}], "jobID": "job_Boss_minsheng-kefu-manual-id"}]}}'

## candidate filter

In [53]:
# boss_candidate_info = json.loads(open('test_candidate.json').read())
# r = requests.post(url+api_dict['filter_candidate'],json={
#     'accountID': account_id,
#     'jobID': job_id,
#     'candidateInfo': boss_candidate_info,
    
# })
# r.text

In [75]:
import glob
for item in glob.glob('*.json'):
    boss_candidate_info = json.loads(open(item).read())
    r = requests.post(url+api_dict['filter_candidate'],json={
        'accountID': 'account_Boss_611930762',
        'candidateInfo': boss_candidate_info
    })
    print(f'item {item} return {r.text}')

item 66140326_周先生.json return {"ret": 0, "msg": "success", "data": {"touch": true}}
item 7320825_范鹏程.json return {"ret": 0, "msg": "success", "data": {"touch": true}}
item 581340655_高伟强.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item 529524709_李扬.json return {"ret": 0, "msg": "success", "data": {"touch": true}}
item 50728376_单子.json return {"ret": 0, "msg": "success", "data": {"touch": true}}
item 550553063_刘骏.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item 520885698_王佳妮.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item 59025635_高女士.json return {"ret": 0, "msg": "success", "data": {"touch": true}}
item 536467792_陈玉如.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item 611727435_古先生.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item 552518369_张银增.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item 107499462_刘德龙.json return {"ret": 0, "msg": "success", "data":

In [77]:
import glob
for item in glob.glob('../success/*.json'):
    boss_candidate_info = json.loads(open(item).read())
    r = requests.post(url+api_dict['filter_candidate'],json={
        'accountID': 'account_Boss_611930762',
        'candidateInfo': boss_candidate_info
    })
    print(f'item {item} return {r.text}')

item ../success/110901299_张坤.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item ../success/549282385_赵雪华.json return {"ret": 0, "msg": "success", "data": {"touch": true}}
item ../success/42038072_蒋静茹.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item ../success/44820815_王婷.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item ../success/611378388_李立根.json return {"ret": 0, "msg": "success", "data": {"touch": true}}
item ../success/114252114_王子涵.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item ../success/518626987_赵玉亮.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item ../success/598295714_陆禹彤.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item ../success/205972415_朱睿.json return {"ret": 0, "msg": "success", "data": {"touch": false}}
item ../success/554846108_刘宵圆.json return {"ret": 0, "msg": "success", "data": {"touch": true}}
item ../success/30750296_李镔钰.json retu

## chat

### search chat

In [38]:
candidate_id='mock-589791359'
candidate_name=candidate_id
history=[{'speaker': 'robot','msg':'这里是一家很牛逼的企业，看上你了，来吧老弟'}, {'speaker':'user','msg':'滚'}]
r = requests.post(url+api_dict['chat_candidate'],json={
    'accountID': account_id,
    'jobID': job_id,
    'candidateID': candidate_id,
    'candidateName': candidate_name,
    'historyMsg': history
})
r.text

'{"ret": 0, "msg": "success", "data": {"nextStep": "normal_chat", "nextStepContent": ""}}'

### user ask chat

In [54]:
candidate_id='mock-589791359'
candidate_name=candidate_id
history=[{'speaker': 'user','msg':'你们这里客服薪资什么样呢'}]
r = requests.post(url+api_dict['chat_candidate'],json={
    'accountID': account_id,
    'jobID': job_id,
    'candidateID': candidate_id,
    'candidateName': candidate_name,
    'historyMsg': history
})
r.text

'{"ret": 0, "msg": "success", "data": {"nextStep": "need_contact", "nextStepContent": "\\u8bd5\\u7528\\u671f\\u4fdd\\u5e955000\\uff0c\\u8f6c\\u6b63\\u540e8000-20000\\uff0c\\u624b\\u7eed\\u8d39\\u63d0\\u62100.5%-1.5%\\u3002\\u54b1\\u80fd\\u4e0d\\u80fd\\u7559\\u4e2a\\u7b80\\u5386\\u548c\\u8054\\u7cfb\\u65b9\\u5f0f\\u5462"}}'

## chat result

In [41]:
candidate_id='mock-589791359'
candidate_name=candidate_id
files = {'cv': open('test_cv.pdf','rb')}
r = requests.post(url+api_dict['result_candidate'], data={
    'accountID': account_id,
    'jobID': job_id,
    'candidateID': candidate_id,
    'candidateName': candidate_name,
    'phone': '139xxxx5556',
    'wechat': '139xxxx5556'
},files=files)
r.text


'{"ret": 0, "msg": "success", "data": {"status": "ok"}}'

In [82]:
files = {'cv': open('test_cv.pdf','rb')}
r = requests.post(url+api_dict['result_candidate'], data={
    'accountID': 'account_Boss_27175761',
    'candidateID': '111735749',
    'candidateName': '李顺东',
},files=files)
r.text

'{"ret": 0, "msg": "success", "data": {"status": "ok"}}'